In [1]:
from load_sms_dataset import *

/Users/jason.xie/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
X_train, X_test, y_train, y_test, vectorizer = load_spam_dataset("~/Downloads/smsspamcollection/SMSSpamCollection")

In [154]:
class NaiveBayesClassifier:
    '''
        A binary classifier - P(y | x) = P(x | y) * P(y) / P(x)
    '''
   
    def fit(self, X, y, vocab=None):
        self.vocab = vocab
        self.priors = np.array([[1 - np.mean(y), np.mean(y)]])
        
        self.likelihood = np.zeros((X.shape[1], 2)) # P(x | y)
        for i in range(X.shape[1]): # go over every word
            arr = X[:,i].todense()
            for label in [0, 1]:
                arr_l = arr[y == label]
                self.likelihood[i, label] = (np.sum(arr_l >= 1) + 1.0) / (np.sum(arr_l >= -1) + 2) # simple smoothing
                # wow this is so efficient
    
    def predict(self, X):
        pred = np.array([self.predictOne(np.array(list(a.flat))) for a in X_test.todense()])
        return pred
        
    def predictOne(self, X):
        LL = np.concatenate((self.priors, self.likelihood[X == 1]), axis=0)
        logp = np.sum(np.log(LL), axis=0) # e.g. [-30, -28]
        return np.exp(logp[1]) / np.sum(np.exp(logp)) # e.g. e^-28 / (e^-30 + e^-28)
        

In [155]:
clf = NaiveBayesClassifier()
clf.fit(X_train, y_train, vocab=vectorizer.vocabulary_inverted)


In [160]:
y_test_pred = clf.predict(X_test)
print y_test_pred
print y_test_pred.shape

[ 0.7906313   0.02442748  0.13862192 ...,  0.17539357  0.99999998
  0.0220277 ]
(1839,)


In [164]:
from sklearn.metrics import accuracy_score

# print accuracy_score(y_test, y_test_pred)
print y_test_pred >= 0.5

[ True False False ..., False  True False]
